# Isolation Levels

## Read Committed Isolation Level

### Specifications

- Select Queries: sees only data committed before the **query** began, easy to achieve with MVCC snapshot.
- DML Queries: blocked by other DML queries on the same rows, wait for them to commit / rollback.\
If they succeed the updated version of the rows that were found are evaluated again against the `WHERE` qualifiers.

### Illustration

#### Simple Usage - May Be OK

Transfer between accounts

```SQL
BEGIN;
UPDATE accounts SET balance = balance + 100.00 WHERE acctnum = 1;
-- run from another session: UPDATE accounts SET balance = balance - 100.00 WHERE acctnum = 1;
UPDATE accounts SET balance = balance - 100.00 WHERE acctnum = 2;
COMMIT;
```

#### Complex Usage - Problematic

Assume website is a two-row table with website.hits equaling 9 and 10:

```SQL
DROP TABLE IF EXISTS read_committed;
CREATE TABLE read_committed(id INT, data INT);
INSERT INTO read_committed VALUES (1, 10), (2, 11);
COMMIT;
```

```SQL
/** Transaction 1 **/
BEGIN;
UPDATE read_committed SET data = data + 1;
```

```SQL
/** Transaction 2 **/
BEGIN;
DELETE FROM read_committed WHERE data = 11;
-- This is blocked because transaction 1 is updating the table
```

```SQL
/** Transaction 1 **/
COMMIT;
```

```SQL
/** Transaction 2 **/
-- Now it's unblocked, which row is deleted?
SELECT * FROM read_committed;
COMMIT;
```

## Repeatable Read Isolation Level

### Specifications

- Classic snapshot isolation.
- Select Queries: sees only data committed before the **first non-transaction-control statement in the transaction** began.
- DML Queries: current transaction is blocked by other DML transactions on the same rows, wait for them to commit / rollback.\
If they succeed the, the transaction is rolled back.
- An app that uses this or more strict level should be prepare to deal with serialization failures -> retry transaction.

### Illustration

```SQL
/** Transaction 1 **/
BEGIN;
UPDATE read_committed SET data = data + 1;
```

```SQL
/** Transaction 2 **/
BEGIN ISOLATION LEVEL REPEATABLE READ;
DELETE FROM read_committed WHERE data = 11;
```

```SQL
/** Transaction 1 **/
COMMIT;
```

```SQL
/** Transaction 2 **/
-- This just finishes with an error
ROLLBACK;
```

## Serializable Isolation Level

- Serializable snapshot isolation.
- Similar to repeatable read, but takes predicate (non-blocking) locks to identify potential read-write dependencies.
- Eliminates any non-serial anomalies, *assuming that all the transactions operating on this data are serializable!*